<a href="https://colab.research.google.com/github/josefrancisco81788/Python-Projects/blob/main/Exploration_Trading_Strategy_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install backtesting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173913 sha256=d23c6631230311f196da0045c05720d5b33c13cd1664a43b0b6a7b6b232967c1
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import GOOG

In [ ]:
# Simple Moving Average (SMA) function
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(window=n, min_periods=1).mean()

# Define the strategy class
class SmaCross(Strategy):
    n1 = 10  # Shorter MA window
    n2 = 20  # Longer MA window

    def init(self):
        """Initialize the indicators to be used in the strategy"""
        # Compute moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        """Define the trading strategy to execute for each step"""
        # If the short-term moving average crosses above the long-term one, buy
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
        # If the short-term moving average crosses below the long-term one, sell
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [4]:
# Backtest the strategy on Google stock data
bt = Backtest(GOOG, SmaCross, cash=10_000, commission=.002)
# Run the backtest and print the statistics
stats = bt.run()
print(stats)
# Plot the performance of the strategy
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/usr/local/lib/python3.10/dist-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-cons

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   99.441341
Equity Final [$]                  86023.67432
Equity Peak [$]                   86992.02432
Return [%]                         760.236743
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    28.72056
Volatility (Ann.) [%]               41.629197
Sharpe Ratio                         0.689914
Sortino Ratio                        1.412292
Calmar Ratio                         0.765188
Max. Drawdown [%]                  -37.533997
Avg. Drawdown [%]                   -5.794104
Max. Drawdown Duration      689 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                   96
Win Rate [%]                        54.166667
Best Trade [%]                       57.11931
Worst Trade [%]                    -16.629898
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for '

GridPlot(id='p1330', ...)

In [5]:
# MACD function
def MACD(values, n_fast=12, n_slow=26, n_signal=9):
    """
    Calculate MACD and Signal Line
    """
    ema_fast = pd.Series(values).ewm(span=n_fast, min_periods=1).mean()
    ema_slow = pd.Series(values).ewm(span=n_slow, min_periods=1).mean()
    macd = ema_fast - ema_slow
    signal = macd.ewm(span=n_signal, min_periods=1).mean()
    return macd, signal

# Define the strategy class
class MacdStrategy(Strategy):
    n_fast = 12  # Fast EMA window
    n_slow = 26  # Slow EMA window
    n_signal = 9  # Signal line window

    def init(self):
        """Initialize the indicators to be used in the strategy"""
        # Compute MACD and Signal line
        self.macd, self.signal = self.I(MACD, self.data.Close, self.n_fast, self.n_slow, self.n_signal)

    def next(self):
        """Define the trading strategy to execute for each step"""
        # If MACD crosses above the Signal line, buy
        if crossover(self.macd, self.signal):
            self.position.close()
            self.buy()
        # If MACD crosses below the Signal line, sell
        elif crossover(self.signal, self.macd):
            self.position.close()
            self.sell()

In [6]:
# Backtest the strategy on Google stock data
bt = Backtest(GOOG, MacdStrategy, cash=10_000, commission=.002)
# Run the backtest and print the statistics
stats = bt.run()
print(stats)
# Plot the performance of the strategy
bt.plot()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    99.81378
Equity Final [$]                  26350.67554
Equity Peak [$]                   28201.02412
Return [%]                         163.506755
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   12.038331
Volatility (Ann.) [%]               37.563831
Sharpe Ratio                         0.320477
Sortino Ratio                         0.55658
Calmar Ratio                         0.276973
Max. Drawdown [%]                  -43.463904
Avg. Drawdown [%]                   -6.458891
Max. Drawdown Duration     1555 days 00:00:00
Avg. Drawdown Duration       91 days 00:00:00
# Trades                                  157
Win Rate [%]                        39.490446
Best Trade [%]                      59.982015
Worst Trade [%]                    -18.874562
Avg. Trade [%]                    

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:455: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df2 = (df.assign(_width=1).set_index('datetime')
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for '

GridPlot(id='p1727', ...)

# Strategy Performance Comparison

| Metric                | MACD Strategy    | SMA Strategy     |
|-----------------------|------------------|------------------|
| Exposure Time [%]     | 99.81%           | 99.44%           |
| Equity Final [$]      | 26,350.68        | 86,023.67        |
| Return [%]            | 163.51%          | 760.24%          |
| Return (Ann.) [%]     | 12.04%           | 28.72%           |
| Sharpe Ratio          | 0.32             | 0.69             |
| Sortino Ratio         | 0.56             | 1.41             |
| Calmar Ratio          | 0.28             | 0.77             |
| Max. Drawdown [%]     | -43.46%          | -37.53%          |
| Trades                | 157              | 96               |
| Win Rate [%]          | 39.49%           | 54.17%           |
| Profit Factor         | 1.41             | 2.27             |
| SQN                   | 0.90             | 2.04             |

# Comparison:

## Performance:
The SMA strategy significantly outperformed the MACD strategy, with a much higher Equity Final ($86,023.67 vs $26,350.68) and Return (%) (760.24% vs 163.51%).

## Risk Metrics:
The Max. Drawdown was smaller for the SMA strategy (-37.53% vs -43.46%), and it also had a higher Calmar Ratio (0.77 vs 0.28), indicating better risk-adjusted returns.

## Sharpe & Sortino Ratios:
The SMA strategy had higher Sharpe (0.69 vs 0.32) and Sortino Ratios (1.41 vs 0.56), which implies that it provided better risk-adjusted returns and was less volatile compared to MACD.

## Win Rate and Profit Factor:
The SMA strategy had a higher Win Rate (54.17% vs 39.49%) and Profit Factor (2.27 vs 1.41), meaning that winning trades were more frequent and profitable.

## Number of Trades:
The MACD strategy executed more trades (157 vs 96), indicating higher trading activity, but it resulted in a lower overall profit compared to the SMA strategy.

# Conclusion:
The SMA strategy performed better overall compared to the MACD strategy, especially in terms of total returns, win rate, and risk-adjusted metrics. The MACD strategy had a larger number of trades but suffered from a higher drawdown and lower profitability.

## The SMA crossover seems more efficient and profitable with lower risk during the backtesting period.
